In [4]:
import random
import copy
import math
import numpy as np
import pyspark
import csv

inn_numbers = 0
inovatii_generatii = {}

def sigmoid(x):
    x = np.array([x])
    return np.mean(1/(1+ np.exp(-4.9 * x)))

class Gena:
    def __init__(self, nume, tip):
        self.nume = nume
        self.tip = tip
    def __eq__(self, gena1):
        if self.nume == gena1.nume:
            return True
        return False
    def __hash__(self):
        return self.nume
    def __str__(self):
        return str(self.nume)
    def __lt__(self, gena_aux):
        if self.nume < gena_aux.nume:
            return True
        else:
            return False

class Conexiune:
    def __init__(self, inp, out, wgh):
        self.input = inp
        self.output = out
        self.weight = wgh
        self.status = 1
        self.innovation_number = self.determinare_inovatie(inp, out)
    def __str__(self):
        return str(self.input) + ", " + str(self.output) + ", " + str(self.weight) + ", " + str(self.innovation_number) + ", " + str(self.status) + " - "
    def __eq__(self, conn):
        if self.innovation_number == conn.innovation_number:
            return True
        else:
            return False
    def __hash__(self):
        return self.innovation_number
    def determinare_inovatie(self, inp, out):
        global inovatii_generatii
        global inn_numbers
        if (inp, out) in inovatii_generatii.values():
            for i in inovatii_generatii:
                if inovatii_generatii[i] == (inp, out):
                    return i
        else:
            inn_numbers += 1
            inovatii_generatii[inn_numbers] = (inp, out)
            return inn_numbers

class Genom:
    def __init__(self, gene, conexiuni):
        self.gene = copy.deepcopy(gene)
        self.conexiuni = copy.deepcopy(conexiuni)
        self.fitness = self.feed_forwoard()
        
    def __str__(self):
        text = "Gene: \n"
        for i in self.gene:
            text = text + str(i) + "; "
        text += "\nConexiuni: \n"
        for i in self.conexiuni:
            text = text + str(i) + "; "
        return text
    
    
    def eliminare_disabled(self):
        de_sters = [n for n in self.conexiuni if n.status == 0]
        for conn in de_sters:
            self.conexiuni.remove(conn)

    
    def adaugare_conexiune(self, i, j):
        verificare_con = 0
        maxim = 0
        for k in range(0, len(self.conexiuni)):
            if self.conexiuni[k].input == i and self.conexiuni[k].output == j:
                if self.conexiuni[k].status == 0:
                    verificare_con = 1
                    maxim = k
                else:
                    return
        if verificare_con == 1:
            self.conexiuni[maxim].status = 1
        else:
            con = Conexiune(i, j, random.uniform(0, 1)*random.choice([-1, 1]))
            self.conexiuni += [con]
        
    def adaugare_gena(self, gena):
        self.gene += [gena]
        
    def modificare_weighturi(self):
        conexiune = random.choice(self.conexiuni)
        verificare = random.uniform(0, 1)
        if verificare <= 0.8:
            conexiune.weight += random.uniform(0, 1) * random.choice([-1, 1])
        else:
            conexiune.weight = random.uniform(0, 1)*random.choice([-1, 1])

            
    def adaugare_nod(self):
        nume_genomi = [i.nume for i in self.gene]
        conexiuni_valide = [i for i in self.conexiuni if i.status]
        if len(conexiuni_valide) == 0:
            return
        conexiune = random.choice(conexiuni_valide)
        conexiune.status = 0
        gena_noua = Gena(max(nume_genomi) + 1, 2)
        conexiune_noua1 = Conexiune(conexiune.input, gena_noua, 1)
        conexiune_noua2 = Conexiune(gena_noua ,conexiune.output, conexiune.weight)
        self.gene += [gena_noua]
        self.conexiuni += [conexiune_noua1, conexiune_noua2]
        
    def mutare(self):
        verificare = random.randint(0, 100)
        if verificare < 90:
            self.modificare_weighturi()
        verificare = random.randint(0, 100)
        if verificare < 5:
            conex = [(i.input, i.output) for i in self.conexiuni]
            pereche = self.random_nodes()
            if pereche not in conex and (pereche[1], pereche[0]) not in conex:
                self.adaugare_conexiune(*pereche)
                
        verificare = random.randint(0, 100)
        if verificare < 3:
            self.adaugare_nod()

            
    def random_nodes(self):
        i = random.choice([n for n in self.gene if n.tip != 1])
        j_list = [n for n in self.gene if n.tip != 0]
        if len(j_list) == 0:
            nume = [n.nume for n in self.gene]
            j = Gena(max(nume) + 1, 2)
            self.adaugare_nod()
        else:
            j = random.choice(j_list)
        return (i, j)
    
    
    def feed_forwoard(self):
        suma = 0
        teste = []
        """with open('C:\\Users\\MrSpV\\Desktop\\data.csv') as csvfile:
            readCSV = csv.reader(csvfile, delimiter=',')
            for row in readCSV:
                if row[0] == 'id':
                    continue
                if row[1] == 'M':
                    teste = teste + [([1] + list(map(lambda x: float(x), row[2:])),[1])]
                else:
                    teste = teste + [([1] + list(map(lambda x: float(x), row[2:])),[0])]
        #teste = teste[1:]"""
        teste = [([1,1,1], [0]), ([1,1,0], [1]), ([1,0,1], [1]), ([1,0,0], [0])]
        for test in teste:
            rezultat = self.ff_util(test[0])
            rezultat = np.mean(np.array(test[1])) - rezultat
            rezultat = rezultat * rezultat
            suma += rezultat
        return suma / 4
    
    def ff(self, inputs):
        valori = {}
        valori1 = {}
        for i in self.gene:
            valori[i.nume] = 0
            valori1[i.nume] = 0
        inp = [n for n in self.gene if n.tip == 0]
        inp.sort()
        for i in range(0, len(inp)):
            valori[inp[i].nume] += inputs[i]
        for conex in self.conexiuni:
            valori[conex.output.nume] += sigmoid(valori[conex.input.nume] * conex.weight)

        for conex in self.conexiuni:
            valori1[conex.output.nume] += sigmoid(valori[conex.input.nume] * conex.weight)
            
        #print(valori)
        out = [n for n in self.gene if n.tip == 1]
        return valori1[out[0].nume]
        
    
    def ff_util(self, inputs):
        valori = []
        inp = [n for n in self.gene if n.tip == 0]
        inp.sort()
        coada = [n for n in self.gene if n.tip == 1]
        for i in range(0, len(inp)):
            valori += [(inp[i], inputs[i])]
        ttl = 2000
        while coada != []:
            if ttl == 0:
                return 1000
            ttl -= 1
            nod = coada.pop()
            if nod.tip == 0:
                continue
            adiacente = []
            conexiuni_adiacente = []
            for i in self.conexiuni:
                if i.output.nume == nod.nume and i.status == 1:
                    adiacente += [i.input]
                    conexiuni_adiacente += [i]
            posibile = [n.nume for n in coada]
            for i in adiacente:
                if i.nume not in posibile:
                    coada += adiacente
            ok = 0
            suma = 0
            for i in range(0, len(adiacente)):
                ok1 = 0
                for j in valori:
                    if j[0].nume == adiacente[i].nume:
                        suma =  suma + j[1] * conexiuni_adiacente[i].weight
                        ok1 = 1
                        break
                if ok1 == 0:
                    ok = 1
                    break
            if ok == 1:
                coada = [nod] + coada
            else:
                ok2 = 0
                for i in range(0, len(valori)):
                    if valori[i][0].nume == nod.nume:
                        ok2 = i
                        break
                if ok2 == 0:
                    valori += [(nod, sigmoid(suma))]
                else:
                    valori[i] = (nod, sigmoid(suma))
        return np.mean(np.array([n[1] for n in valori if n[0].tip == 1]))
    
    def feed_forwoard_util(self, inputs):
        valoare_noduri = []
        coada = []
        inputuri = [n for n in self.gene if n.tip == 0]
        coada = [n for n in self.gene if n.tip == 1]
        for i in range(0, len(inputuri)):
            valoare_noduri += [(inputuri[i], inputs[i])]
        while coada != []:
            nod = coada.pop()
            if nod.tip == 0:
                continue
            adiacente = []
            conexiuni_adiacente = []
            for i in self.conexiuni:
                if i.output.nume == nod.nume:
                    adiacente += [i.input]
                    conexiuni_adiacente += [i]
            coada += adiacente
            coada = list(set(coada))
            ok = 0
            suma = 0
            for i in range(0, len(adiacente)):
                ok1 = 0
                for j in valoare_noduri:
                    if j[0].nume == adiacente[i].nume:
                        suma += j[1] * conexiuni_adiacente[i].weight
                        ok1 = 1
                        break
                if ok1 == 0:
                    ok = 1
                    break
            if ok == 1:
                coada = [nod] + coada
            else:
                ok2 = 0
                for i in range(0, len(valoare_noduri)):
                    if valoare_noduri[i][0].nume == nod.nume:
                        ok2 = i
                        break
                if ok2 == 0:
                    valoare_noduri += [(nod, suma)]
                else:
                    valoare_noduri[i] = (nod, suma)
        return np.mean(np.array([n[1] for n in valoare_noduri if n[0].tip == 1]))
    
    def clonare(self):
        clona = Genom(self.gene, self.conexiuni)
        return clona

 
class Specie:
    def __init__(self, genomi):
        self.genomi = copy.deepcopy(genomi)
        self.max_fitness = self.determinare_campion()
        self.max_generatie = self.determinare_campion()
        self.generatie_curenta = 1
        self.moarta = 0
        
    def determinare_superior(self):
        maxim = -100
        for genom in self.genomi:
            fitness = genom.fitness
            if fitness > maxim:
                maxim = fitness
        if maxim > self.max_fitness:
            self.max_fitness = maxim
            self.max_generatie = self.generatie_curenta
            
            
    def determinare_campion(self):
        maxim = -100
        for genom in self.genomi:
            fitness = genom.fitness
            if fitness > maxim:
                maxim = fitness
        return maxim
    
    def determinare_existenta(self):
        if self.generatie_curenta == 1:
            return True
        fitness = self.determinare_campion()
        self.determinare_superior()
        fit = fitness  != self.max_fitness
        end = self.generatie_curenta != self.max_generatie
        return fit and end
    
    def breed(self):
        if random.uniform(0, 1) < 0.4 or len(self.genomi) == 1:
            child = random.choice(self.genomi).clonare()
            child.mutare()
        else:
            mom = random.choice(self.genomi)
            dad = random.choice([i for i in self.genomi if i != mom])
            if mom.fitness < dad.fitness:
                child = self.crossover(mom, dad)
            else:
                child = self.crossover(dad, mom)
        return child
    
            
    def eliminare_genomi(self, tip):
        ok = 1
        ttl = 300
        while ok == 1 and ttl > 0:
            ttl -= 1
            ok = 0
            for i in range(0, len(self.genomi) - 1):
                if self.genomi[i].fitness > self.genomi[i + 1].fitness:
                    self.genomi[i], self.genomi[i + 1] = self.genomi[i + 1], self.genomi[i]
                    ok = 1
        if tip == 1:
            self.genomi = self.genomi[0]
        else:
            self.genomi = self.genomi[:math.ceil(len(self.genomi) / 4)]
        
    def fitness_specie(self):
        suma = 0
        for i in self.genomi:
            suma += i.fitness
        return suma
    
    def genom_maxim(self):
        maxim = 1000
        genom_maxim = 0
        for genom in self.genomi:
            if genom.fitness < maxim:
                maxim = genom.fitness
                genom_maxim = genom
        return (maxim, genom_maxim)
    
    def crossover(self, genom1, genom2):
        lista_gene = []
        for i in genom1.gene:
            if i not in genom2.gene:
                lista_gene += [i]
        lista_gene += genom2.gene
        conexiuni = []
        for i in genom1.conexiuni:
            ok = 0
            for j in genom2.conexiuni:
                if i.input == j.input and i.output == j.output:
                    if random.choice([0, 1]) == 0:
                        conexiuni.append(copy.deepcopy(i))
                    else:
                        conexiuni.append(copy.deepcopy(j))
                    ok = 1
                    break
            if ok == 0:
                conexiuni.append(i)
        de_sters = []
        for i in lista_gene:
            if i.nume not in [n.input.nume for n in conexiuni] + [n.output.nume for n in conexiuni]:
                de_sters += [i]
        for i in de_sters:
            lista_gene.remove(i)
        copil = Genom(lista_gene, conexiuni)
        return copil
    
    def distanta_genomi(self, genom1, genom2):
        numere1 = [n.innovation_number for n in genom1.conexiuni]
        numere2 = [n.innovation_number for n in genom2.conexiuni]
        maxn1 = max(numere1)
        maxn2 = max(numere2)
        prag = 3
        d = len([n for n in numere1 if n not in numere2 and n < maxn2] + [n for n in numere2 if n not in numere1 and n < maxn1])
        e = len([n for n in numere1 if n not in numere2 and n > maxn2] + [n for n in numere2 if n not in numere1 and n > maxn1])
        c = 0.4
        n = max(len(numere1), len(numere2))
        w = abs(genom1.fitness - genom2.fitness)
        if (d + e) / n + c * w < prag:
            return 1
        else:
            return 0
        

        
class Specii:
    def __init__(self, spe):
        self.spec = spe
        
    def update_fitness(self):
        for specie in self.spec:
            if specie.genom_maxim()[0] < specie.max_fitness:
                specie.max_fitness = specie.genom_maxim()[0]
                specie.max_generatie = specie.generatie_curenta
                
    def eliminare_specii(self):
        genab = Gena(0, 0)
        gena1 = Gena(1, 0)
        gena2 = Gena(2, 0)
        gena3 = Gena(3, 2)
        gena0 = Gena(5, 2)
        gena4 = Gena(4, 1)
        gena5 = Gena(4, 1)
        obj =  Conexiune(gena1, gena4, random.uniform(0, 1)*random.choice([-1, 1]))
        ob2 =  Conexiune(gena2, gena4, random.uniform(0, 1)*random.choice([-1, 1]))
        ob3 =  Conexiune(genab, gena4, random.uniform(0, 1)*random.choice([-1, 1]))
        gen1 = Genom([genab,gena1, gena2, gena4], [obj, ob2, ob3])

        lista1 = []
        for _ in range(0, 15):
            lista1.append(copy.deepcopy(gen1))
        self.update_fitness()
                
        de_sters = []
        for i in range(0, len(self.spec)):
            if self.spec[i].generatie_curenta - self.spec[i].max_generatie >= 15:
                de_sters += [i]
        for i in de_sters:
            self.spec[i].genomi = lista1
                
        
    def clasificare_genom(self, genom):
        start_time = time.time()
        if len(self.spec) == 0:
            self.spec += [Specie([genom])]
        else:
            for specie in self.spec:
                if len(specie.genomi) == 0:
                    continue
                if self.distanta_genomi(specie.genomi[0], genom) == 1 and len(specie.genomi) < 15:
                    specie.genomi += [genom]
                    #print("--- %s por RIO ---" % (time.time() - start_time))
                    return
            self.spec += [Specie([genom])]
            
    def distanta_genomi(self, genom1, genom2):
        numere1 = [n.innovation_number for n in genom1.conexiuni]
        numere2 = [n.innovation_number for n in genom2.conexiuni]
        maxn1 = max(numere1)
        maxn2 = max(numere2)
        prag = 3
        d = len([n for n in numere1 if n not in numere2 and n < maxn2] + [n for n in numere2 if n not in numere1 and n < maxn1])
        e = len([n for n in numere1 if n not in numere2 and n > maxn2] + [n for n in numere2 if n not in numere1 and n > maxn1])
        c = 0.4
        n = max(len(numere1), len(numere2))
        w = abs(genom1.fitness - genom2.fitness)
        if (d + e) / n + c * w < prag:
            return 1
        else:
            return 0
        
    
    def eliminare_underfit_specii(self, tip):
        for i in self.spec:
            i.eliminare_genomi(tip)
    
    def fitness_specii(self):
        suma = 0
        for i in self.spec:
            suma += i.fitness_specie()
        return suma
    
    def determinare_populatie(self):
        return sum([len(n.genomi) for n in self.spec])
    
    def specie_maximala(self):
        minim = 1000
        specie_minima = 0
        genom_minim = 0
        for specie in self.spec:
            if len(specie.genomi) == 0:
                continue
            if specie.genom_maxim()[0] < minim:
                minim = specie.genom_maxim()[0]
                specie_minima = specie
                genom_minim = specie.genom_maxim()[1]
        return minim
            
            
    def evolutie(self):
        pop_finala = self.determinare_populatie()
        lungime_populatie = [len(n.genomi) for n in self.spec]
        k = -1
        self.eliminare_underfit_specii(0)
        copii = []
        for specie in self.spec:
            k += 1
            ratio = specie.fitness_specie() / self.fitness_specii()
            offspring = math.floor(ratio * (pop_finala-lungime_populatie[k]) - 2)
            offspring = 11
            for j in range(0, int(offspring)):
                copii.append(specie.breed())
        for copil in copii:
            self.clasificare_genom(copil)
        for specie in self.spec:
            specie.generatie_curenta += 1

        
    
genab = Gena(0, 0)
gena1 = Gena(1, 0)
gena2 = Gena(2, 0)
gena3 = Gena(3, 2)
gena0 = Gena(5, 2)
gena4 = Gena(4, 1)
obj =  Conexiune(gena1, gena4, random.uniform(0, 1)*random.choice([-1, 1]))
ob2 =  Conexiune(gena2, gena4, random.uniform(0, 1)*random.choice([-1, 1]))
ob3 =  Conexiune(genab, gena4, random.uniform(0, 1)*random.choice([-1, 1]))
ob4 =  Conexiune(gena1, gena3, random.uniform(0, 1)*random.choice([-1, 1]))
ob5 =  Conexiune(gena2, gena3, random.uniform(0, 1)*random.choice([-1, 1]))
ob6 =  Conexiune(genab, gena3, random.uniform(0, 1)*random.choice([-1, 1]))
ob7 =  Conexiune(gena3, gena4, random.uniform(0, 1)*random.choice([-1, 1]))
ob8 = Conexiune(gena3, gena4, random.uniform(0, 1)*random.choice([-1, 1]))
gen1 = Genom([genab,gena1, gena2, gena4], [obj, ob2, ob3])
gen2 = Genom([genab,gena1, gena2,gena3, gena4], [ob4, ob5, ob6, ob7])
gen3 = Genom([genab,gena1, gena2,gena3, gena4], [obj, ob2, ob4, ob7])
gen4 = Genom([genab,gena1, gena2, gena3, gena4], [obj, ob3, ob5, ob7])
gen5 = Genom([genab,gena1, gena2,gena3, gena4], [ob6, ob2, ob3, ob7])

lista1 = []
for _ in range(0, 15):
    #lista1.append(copy.deepcopy(random.choice([ genom_special])))
    lista1.append(copy.deepcopy(random.choice([gen1, gen2, gen3, gen4, gen5])))
specie1= Specie(copy.deepcopy(lista1))
sp = []
for _ in range(0, 10):
    sp = sp + [copy.deepcopy(specie1)]
spc = Specii(copy.deepcopy(sp))
            



In [5]:
import time




precedent = 1000
epoci = 150
for i in range(0, len(spc.spec)):
    for genom in spc.spec[i].genomi:
        genom.fitness = genom.feed_forwoard()
actual = 100
while actual > 0.05:
    spc.eliminare_specii()
    actual = min([n.genom_maxim()[0] for n in spc.spec])
    print(actual)
    print(epoci)
    print([len(n.genomi) for n in spc.spec])
    #print([len(n.genomi) for n in spc.spec])
    start_time = time.time()
    epoci -= 1
    if precedent >= actual:
        for i in range(0, len(spc.spec)):
            for genom in spc.spec[i].genomi:
                genom.mutare()
    for i in range(0, len(spc.spec)):
        for genom in spc.spec[i].genomi:
            genom.fitness = genom.feed_forwoard()
    spc.evolutie()
    precedent = actual
    print("--- %s seconds iteratie---" % (time.time() - start_time))
for i in range(0, len(spc.spec)):
    for genom in spc.spec[i].genomi:
        genom.eliminare_disabled()
        genom.fitness = genom.feed_forwoard()

0.2508821054336964
150
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
--- 0.15558457374572754 seconds iteratie---
0.2383964919592258
149
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
--- 0.17956137657165527 seconds iteratie---
0.22285662808973347
148
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
--- 0.7968275547027588 seconds iteratie---
0.20295677548912006
147
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
--- 0.22739195823669434 seconds iteratie---
0.20421312758823057
146
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
--- 0.1506359577178955 seconds iteratie---
0.19820564981616337
145
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
--- 0.1685643196105957 seconds iteratie---
0.19665579493887203
144
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
--- 0.2752053737640381 seconds iteratie---
0.19454242679248782
143
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
--- 0.30524396896362305 seconds iteratie---
0.19376587759551905
142
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
--- 0.2513096332550049 seconds iteratie---
0.1937400378998939
141
[15

In [7]:
print([n.genom_maxim()[0] for n in spc.spec])
print(spc.spec[0].genom_maxim()[1])

[0.006305865993467862, 0.12500001354835744, 0.1250000053960178, 0.25000000046880994, 0.06250800027583543, 0.18750001128203564, 0.12500000557456775, 0.18750000000012662, 0.16684478475997902, 0.12500000142224055]
Gene: 
3; 0; 1; 2; 4; 5; 
Conexiuni: 
1, 3, 5.716821241806853, 4, 1 - ; 2, 3, -4.598292835208927, 5, 1 - ; 0, 3, -3.877397503087535, 6, 1 - ; 3, 4, 3.7116979942793273, 7, 1 - ; 1, 4, -1.7505150664805247, 1, 1 - ; 0, 5, -0.08661280941240612, 13, 1 - ; 5, 4, -0.93667018338796, 9, 1 - ; 2, 4, 0.8834304564802042, 2, 1 - ; 


In [10]:
import numpy as np
import cv2



image = cv2.imread('C:\\Users\\MrSpV\\Desktop\\cancer1.jpg') #The function to read from an image into OpenCv is imread()

#imshow() is the function that displays the image on the scre0en.
#The first value is the title of the window, the second is the image file we have previously read.
#cv2.imshow("OpenCV Image Reading", image)
print(len(image) * len(image[0]))

44775


In [30]:
from matplotlib.image import imread
image = imread('C:\\Users\\MrSpV\\Desktop\\cancer2.jpeg')
lst = []
for i in range(1, len(image)):
    lst = lst + list(image[i])
        
print(len(lst))


44352


In [134]:
nod_output = Gena(1,1)
bias = Gena(0, 0)
conn_bias = Conexiune(bias, nod_output, random.uniform(0, 1)*random.choice([-1, 1]))
maxim = 2
gene = [bias, nod_output]
conexiuni = [conn_bias]
for _ in range(0, 30):
    gena_noua = Gena(maxim,0)
    maxim += 1
    conn_noua = Conexiune(gena_noua, nod_output, random.uniform(0, 1)*random.choice([-1, 1]))
    conexiuni += [conn_noua]
    gene += [gena_noua]
genom_special = Genom(gene, conexiuni)
print(genom_special.ff([1]*31))

{0: 1, 1: 0, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1}
None


In [131]:
for i in range(0, len(spc.spec)):
    for genom in spc.spec[i].genomi:
        genom.mutare()

In [130]:
start_time = time.time()
for i in range(0, len(spc.spec)):
    for genom in spc.spec[i].genomi:
        genom.fitness = genom.feed_forwoard()
print(time.time() - start_time)

56.788193225860596
